### DESPLIEGUE


In [16]:
import tkinter as tk
from tkinter import messagebox
import numpy as np
import pickle as pk

Cargamos el modelo entrenado almacenado en el archivo pickle

In [ ]:
with open("C:/Users/Juan Pablo/OneDrive - Universidad EAFIT/Documentos/Eafit/5to Semestre IM/Mode 4/Proyecto final/modelo_final.pickle", 'rb') as f:
    modelo = pk.load(f)

Definimos la función con la que verificamos el correcto formato de las variables que ingresa el usuario

In [18]:
def validar_entradas():
    try:
        # Validar fecha como int con 8 dígitos (YYYYMMDD)
        date = int(entry_date.get())
        if not (20000101 <= date <= 20241231):
            return False  # Fecha fuera de rango válido (sintácticamente)

        # Validar zip code (5 dígitos, entre 50001 y 52809)
        zip_code = int(entry_zip_code.get())
        if not (50001 <= zip_code <= 52809):
            return False

        # Validar county_number entre 1 y 99
        county_number = float(entry_county_number.get())
        if not (1 <= county_number <= 99):
            return False

        # Validar store_number, pack, bottle_volume, bottles_sold, vendor_number > 0
        store_number = int(entry_store_number.get())
        pack = int(entry_pack.get())
        bottle_volume = int(entry_bottle_volume_ml.get())
        bottles_sold = int(entry_bottles_sold.get())
        vendor_number = float(entry_vendor_number.get())

        if store_number <= 0 or pack <= 0 or bottle_volume <= 0:
            return False
        if bottles_sold <= 0 or vendor_number <= 0:
            return False

        state_bottle_cost = float(entry_state_bottle_cost.get())
        if state_bottle_cost <= 0:
            return False

        # Validar que se haya seleccionado una categoría de whisky
        if whisky_var.get() == "":
            return False

        return True  # Todo válido
    except ValueError:
        return False  # Algún campo no es convertible al tipo requerido


In [19]:
# Habilita o deshabilita el botón de predicción
def habilitar_botón_prediccion():
    if validar_entradas():
        btn_prediccion.config(state=tk.NORMAL)
    else:
        btn_prediccion.config(state=tk.DISABLED)

Definimos la función con la que obtenemos los datos del usuario

In [20]:
def obtener_entradas_usuario():
    try:
        date = int(entry_date.get())
        store_number = int(entry_store_number.get())
        zip_code = int(entry_zip_code.get())
        county_number = float(entry_county_number.get())
        pack = int(entry_pack.get())
        bottle_volume = int(entry_bottle_volume_ml.get())
        bottle_cost = float(entry_state_bottle_cost.get())
        bottles_sold = int(entry_bottles_sold.get())
        vendor_number = float(entry_vendor_number.get())
        

        # Variables categóricas
        blended = whisky_var.get() == "Category_BLENDED WHISKIES"
        canadian = whisky_var.get() == "Category_CANADIAN WHISKIES"
        irish = whisky_var.get() == "Category_IRISH WHISKIES"
        scotch = whisky_var.get() == "Category_SCOTCH WHISKIES"
        straight = whisky_var.get() == "Category_STRAIGHT BOURBON WHISKIES"
        tennessee = whisky_var.get() == "Category_TENNESSEE WHISKIES"

        entradas = [
            date, store_number, zip_code, county_number, pack, 
            bottle_volume, bottle_cost, bottles_sold, vendor_number,
            blended, canadian, irish, scotch, straight, tennessee
        ]

        return np.array(entradas).reshape(1, -1)
    except ValueError:
        messagebox.showerror("Error", "Por favor ingrese valores válidos en todos los campos.")
        return None

Función que ejecuta la predicción

In [21]:
def hacer_prediccion():
    entradas_usuario = obtener_entradas_usuario()
    if entradas_usuario is not None:
        prediccion = modelo.predict(entradas_usuario)
        messagebox.showinfo("Resultado", f"La predicción del precio de venta es: {prediccion[0]:.2f} dolares")

Interfaz

In [22]:
ventana = tk.Tk()
ventana.title("Predicción de Ventas de Whisky")

# Campos de entrada
tk.Label(ventana, text="Fecha (YYYYMMDD):").grid(row=0, column=0, padx=10, pady=5)
entry_date = tk.Entry(ventana)
entry_date.grid(row=0, column=1, padx=10, pady=5)

tk.Label(ventana, text="Número de tienda:").grid(row=1, column=0, padx=10, pady=5)
entry_store_number = tk.Entry(ventana)
entry_store_number.grid(row=1, column=1, padx=10, pady=5)

tk.Label(ventana, text="Código Postal:").grid(row=2, column=0, padx=10, pady=5)
entry_zip_code = tk.Entry(ventana)
entry_zip_code.grid(row=2, column=1, padx=10, pady=5)

tk.Label(ventana, text="Número de condado:").grid(row=3, column=0, padx=10, pady=5)
entry_county_number = tk.Entry(ventana)
entry_county_number.grid(row=3, column=1, padx=10, pady=5)

tk.Label(ventana, text="Pack:").grid(row=4, column=0, padx=10, pady=5)
entry_pack = tk.Entry(ventana)
entry_pack.grid(row=4, column=1, padx=10, pady=5)

tk.Label(ventana, text="Volumen de botella (ml):").grid(row=5, column=0, padx=10, pady=5)
entry_bottle_volume_ml = tk.Entry(ventana)
entry_bottle_volume_ml.grid(row=5, column=1, padx=10, pady=5)

tk.Label(ventana, text="Costo por botella (estado):").grid(row=6, column=0, padx=10, pady=5)
entry_state_bottle_cost = tk.Entry(ventana)
entry_state_bottle_cost.grid(row=6, column=1, padx=10, pady=5)

tk.Label(ventana, text="Botellas vendidas:").grid(row=7, column=0, padx=10, pady=5)
entry_bottles_sold = tk.Entry(ventana)
entry_bottles_sold.grid(row=7, column=1, padx=10, pady=5)

tk.Label(ventana, text="Número de proveedor:").grid(row=8, column=0, padx=10, pady=5)
entry_vendor_number = tk.Entry(ventana)
entry_vendor_number.grid(row=8, column=1, padx=10, pady=5)

# Menú desplegable para tipo de whisky
tk.Label(ventana, text="Tipo de Whisky:").grid(row=9, column=0, padx=10, pady=5)
whisky_var = tk.StringVar(value="")
whisky_menu = tk.OptionMenu(
    ventana, whisky_var,
    "Category_BLENDED WHISKIES",
    "Category_CANADIAN WHISKIES",
    "Category_IRISH WHISKIES",
    "Category_SCOTCH WHISKIES",
    "Category_STRAIGHT BOURBON WHISKIES",
    "Category_TENNESSEE WHISKIES"
)
whisky_menu.grid(row=9, column=1, padx=10, pady=5)

# Botón para predecir
btn_prediccion = tk.Button(ventana, text="Hacer Predicción", command=hacer_prediccion, state=tk.DISABLED)
btn_prediccion.grid(row=10, column=0, columnspan=2, pady=10)

# Eventos para activar botón
campos = [
    entry_date, entry_store_number, entry_zip_code,
    entry_county_number, entry_pack, entry_bottle_volume_ml, entry_state_bottle_cost,
    entry_bottles_sold, entry_vendor_number 
]

for campo in campos:
    campo.bind("<KeyRelease>", lambda e: habilitar_botón_prediccion())

whisky_var.trace("w", lambda *args: habilitar_botón_prediccion())


'2469810813376<lambda>'

Ejecutamos la aplicación

In [23]:
ventana.mainloop()

C:\Users\Juan Pablo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
